In [26]:
import numpy as np 
import pandas as pd 
import os
import os
import zipfile
import pandas as pd
import numpy as np
import csv
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import shutil

In [27]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Training_Set/Training_Set/RFMiD_Training_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Training')

FileExistsError: [Errno 17] File exists: './Training'

In [ ]:
parent_dir="./Training"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [ ]:
save_path = './Training/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)
save_path = './Training/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Training_Set/Training_Set/Training/'+filename,save_path)

In [ ]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/RFMiD_Validation_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Validation')

In [ ]:
parent_dir="./Validation"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [ ]:
save_path = './Validation/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/'+filename,save_path)
save_path = './Validation/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Evaluation_Set/Evaluation_Set/Validation/'+filename,save_path)

In [ ]:
a=[]
b=[]
with open('../input/retinal-disease-classification/Test_Set/Test_Set/RFMiD_Testing_Labels.csv','r') as F:
  reader=csv.reader(F)
  for row in reader:
      if(row[1]=='1'):
        a.append(row[0]+'.png')
      if(row[1]=='0'):
        b.append(row[0]+'.png')
F.close() 
os.mkdir('./Testing')

In [ ]:
parent_dir="./Testing"
directory1 = "Abnormal" 
directory2 = "Normal"
path1 = os.path.join(parent_dir, directory1)
path2 = os.path.join(parent_dir, directory2)
os.mkdir(path1)
os.mkdir(path2)

In [ ]:
save_path = './Testing/Abnormal'
for filename in a:
  shutil.copy('../input/retinal-disease-classification/Test_Set/Test_Set/Test/'+filename,save_path)
save_path = './Testing/Normal'
for filename in b:
  shutil.copy('../input/retinal-disease-classification/Test_Set/Test_Set/Test/'+filename,save_path)

In [ ]:
train_abnormal_dir = os.path.join('./Training/Abnormal')
train_normal_dir = os.path.join('./Training/Normal')
train_abnormal_names = os.listdir(train_abnormal_dir)
train_normal_names = os.listdir(train_normal_dir)
print('total training images with abnormalities:', len(os.listdir(train_abnormal_dir)))
print('total training images of normal eyes:', len(os.listdir(train_normal_dir)))

In [ ]:
validation_abnormal_dir = os.path.join('./Validation/Abnormal')
validation_normal_dir = os.path.join('./Validation/Normal')
validation_abnormal_names = os.listdir(validation_abnormal_dir)
validation_normal_names = os.listdir(validation_normal_dir)
print('total validation images with abnormalities:', len(os.listdir(validation_abnormal_dir)))
print('total validation images of normal eyes:', len(os.listdir(validation_normal_dir)))

In [ ]:
test_abnormal_dir = os.path.join('./Testing/Abnormal')
test_normal_dir = os.path.join('./Testing/Normal')
test_abnormal_names = os.listdir(test_abnormal_dir)
test_normal_names = os.listdir(test_normal_dir)
print('total testing images with abnormalities:', len(os.listdir(test_abnormal_dir)))
print('total testing images of normal eyes:', len(os.listdir(test_normal_dir)))

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                    rotation_range = 30,
                                    width_shift_range = 0.1,
                                    height_shift_range = 0.1,
                                    zoom_range = 0.1,
                                    horizontal_flip = True
                                   )
train_generator = train_datagen.flow_from_directory('./Training', 
                                                    target_size=(150,150), 
                                                    class_mode='binary')

In [ ]:
validation_datagen = ImageDataGenerator(rescale=1/255)
validation_generator = validation_datagen.flow_from_directory('./Validation', 
                                                              target_size=(150,150),
                                                              class_mode='binary')

In [ ]:
test_datagen = ImageDataGenerator(rescale=1/255)
test_generator = test_datagen.flow_from_directory('./Testing', 
                                                              target_size=(150,150),
                                                              class_mode='binary')

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Flatten,Dense,Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow import keras

In [18]:
model = EfficientNetB0(input_shape=(150,150,3), weights='imagenet', include_top=False)

16719872/16705208 [==============================] - 0s 0us/step


In [20]:
for layer in model.layers:
    layer.trainable = False

In [21]:
x = model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

In [28]:
m = Model(inputs=model.input, outputs=predictions)

In [23]:
m.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

In [24]:
results = m.fit(train_generator,epochs=10,
                              steps_per_epoch=20,
                              validation_data=validation_generator,
                             validation_steps=5)

Epoch 1/10
20/20 [==============================] - 112s 5s/step - loss: 1.6095 - accuracy: 0.7359 - val_loss: 0.5915 - val_accuracy: 0.7937
Epoch 2/10
20/20 [==============================] - 98s 5s/step - loss: 0.5700 - accuracy: 0.7641 - val_loss: 0.5773 - val_accuracy: 0.7937
Epoch 3/10
20/20 [==============================] - 97s 5s/step - loss: 0.6223 - accuracy: 0.7312 - val_loss: 0.5372 - val_accuracy: 0.7750
Epoch 4/10
20/20 [==============================] - 100s 5s/step - loss: 0.5540 - accuracy: 0.7703 - val_loss: 0.5781 - val_accuracy: 0.7937
Epoch 5/10
20/20 [==============================] - 103s 5s/step - loss: 0.5296 - accuracy: 0.8109 - val_loss: 0.5266 - val_accuracy: 0.7812
Epoch 6/10
20/20 [==============================] - 100s 5s/step - loss: 0.5772 - accuracy: 0.7672 - val_loss: 0.4887 - val_accuracy: 0.8125
Epoch 7/10
20/20 [==============================] - 101s 5s/step - loss: 0.5708 - accuracy: 0.7469 - val_loss: 0.6809 - val_accuracy: 0.7875
Epoch 8/10
20/2

In [25]:
m.evaluate(test_generator)

20/20 [==============================] - 84s 4s/step - loss: 0.5501 - accuracy: 0.7906


[0.5500901341438293, 0.7906249761581421]